# Creating supercells with pymatgen

The [Pymatgen](https://pymatgen.org/) python library allows to setup
solid-state calculations using a flexible set of classes as well as an API
to an online data base of structures. Its `Structure` and `Lattice`
objects are directly supported by the DFTK `load_atoms` and `load_lattice`
functions, such that DFTK may be readily used to run calculation on systems
defined in pymatgen. Using the `pymatgen_structure` function a conversion
from DFTK to pymatgen structures is also possible. In the following we
use this to create a silicon supercell and find its LDA ground state
using direct minimisation.

First we setup the silicon lattice in DFTK.

In [1]:
using DFTK

a = 10.263141334305942  # Lattice constant in Bohr
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms = [Si => [ones(3)/8, -ones(3)/8]];

Next we make a `[2, 2, 2]` supercell using pymatgen

In [2]:
pystruct = pymatgen_structure(lattice, atoms)
pystruct.make_supercell([2, 2, 2])
lattice = load_lattice(pystruct)
atoms = [Si => [s.frac_coords for s in pystruct.sites]];

Setup an LDA model and discretize using
a single kpoint and a small `Ecut` of 5 Hartree.
Notice that `PlaneWaveBasis` defaults to just using the ``Γ``-point
if no ``k``-point options are provided.

In [3]:
model = model_LDA(lattice, atoms)
basis = PlaneWaveBasis(model, 5)

PlaneWaveBasis (Ecut=5.0, 1 kpoints)

Find the ground state using direct minimisation (always using SCF is boring ...)

In [4]:
scfres = direct_minimization(basis, tol=1e-5);

Iter     Function value   Gradient norm 
     0     1.126336e+02     1.596965e+00
 * time: 0.13038206100463867
     1     1.074902e+01     9.507850e-01
 * time: 1.0014240741729736
     2    -1.201263e+01     1.025653e+00
 * time: 1.1355741024017334
     3    -3.412025e+01     7.958712e-01
 * time: 1.2972631454467773
     4    -4.746779e+01     6.137705e-01
 * time: 1.450796127319336
     5    -5.676820e+01     2.258002e-01
 * time: 1.598973035812378
     6    -5.961201e+01     2.075028e-01
 * time: 1.7069971561431885
     7    -6.080662e+01     9.167877e-02
 * time: 1.8238391876220703
     8    -6.134973e+01     5.030177e-02
 * time: 1.9320850372314453
     9    -6.167378e+01     4.259078e-02
 * time: 2.047516107559204
    10    -6.187269e+01     3.216241e-02
 * time: 2.1570730209350586
    11    -6.199318e+01     2.725707e-02
 * time: 2.273576021194458
    12    -6.207678e+01     1.855462e-02
 * time: 2.3837850093841553
    13    -6.213391e+01     1.563177e-02
 * time: 2.4996240139007

In [5]:
scfres.energies

Energy breakdown:
    Kinetic             25.7671056
    AtomicLocal         -18.8557631
    AtomicNonlocal      14.8522622
    Ewald               -67.1831486
    PspCorrection       -2.3569765
    Hartree             4.8485353 
    Xc                  -19.3336813

    total               -62.261666461579
